    El archivo homicidios viene con dos hojas de excel que son victimas y hechos, abro cada una en un notebook diferente para modificar y tranformar los valores que se necesite

In [ ]:
Se incluye en esta basde de datos cualquier persona muerta ó herida como consecuencia de un siniestro vial

In [1]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime

In [2]:
Archivo_victimas_excel = r"C:\Users\Usuario\Henry\PI2_DA\Consigna\homicidios.xlsx"

df_victmas = pd.read_excel(r"C:\Users\Usuario\Henry\PI2_DA\Consigna\homicidios.xlsx", sheet_name='VICTIMAS')

In [7]:
df_victmas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [4]:
df_victmas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [6]:
df_victmas.size

7170